# Updated taxonomic list from CAPS
This data was provided by Renee as an excel file, we will read the data in ***R*** using package _readxl_ and then write a table into the Postgres database using package _RPostgreSQL_.

In [2]:
require("RPostgreSQL")
require(readxl)
require(dplyr)

Loading required package: RPostgreSQL

Loading required package: DBI

Loading required package: readxl

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




To connect to the database we read the credentials from a file in folder _secrets_.

In [3]:
if (file.exists("../secrets/database.ini")) {
  tmp <- readLines("../secrets/database.ini")
  tmp <- data.frame(strsplit(tmp,"="))
  dbinfo <- tmp[2,]
  names(dbinfo) <- tmp[1,]
   rm(tmp)
} else {
  cat("No database information found")
}

In [4]:
drv <- dbDriver("PostgreSQL") ## remember to update .pgpass file
con <- dbConnect(drv, dbname = dbinfo$database,
                 host = dbinfo$host, port = dbinfo$port,
                 user = dbinfo$user)

Now we read the spreadsheet in folder data:

In [4]:
dts <- read_excel("../data/vis-survey-datasheet-6000.PowerQuery.20210708.xlsx",sheet=1)

Warning message in read_fun(path = enc2native(normalizePath(path)), sheet_i = sheet, :
“Expecting logical in U3675 / R3675C21: got 'Smooth Pest Pear'”
Warning message in read_fun(path = enc2native(normalizePath(path)), sheet_i = sheet, :
“Expecting logical in U12445 / R12445C21: got 'bog pine, creeping pine, dwarf mountain pine, mugo pine, mountain pine, scrub mountain pine, Swiss mountain pine'”
Warning message in read_fun(path = enc2native(normalizePath(path)), sheet_i = sheet, :
“Expecting logical in U12757 / R12757C21: got 'Axebreaker, Capivi'”
Warning message in read_fun(path = enc2native(normalizePath(path)), sheet_i = sheet, :
“Expecting logical in U14944 / R14944C21: got 'Bay Tree, Sweet Bay'”
Warning message in read_fun(path = enc2native(normalizePath(path)), sheet_i = sheet, :
“Expecting logical in U14946 / R14946C21: got 'Pink Agapanthus, Silver Lace'”
Warning message in read_fun(path = enc2native(normalizePath(path)), sheet_i = sheet, :
“Expecting logical in U14949 / R14949

In [5]:
 colnames(dts)

[1] "speciesID"                     "taxonRank"                    
 [3] "classID"                       "class"                        
 [5] "orderID"                       "order"                        
 [7] "familyID"                      "family"                       
 [9] "sortOrder"                     "genusID"                      
[11] "genus"                         "parentSpeciesID"              
[13] "specificEpithet"               "infraspecificEpithet"         
[15] "scientificNameAuthorship"      "scientificNameID"             
[17] "speciesCode_Synonym"           "scientificName"               
[19] "scientificNameHTML"            "vernacularName"               
[21] "otherVernacularNames"          "taxonID"                      
[23] "currentScientificNameCode"     "currentScientificName"        
[25] "currentVernacularName"         "isCurrent"                    
[27] "generalTypeID"                 "generalType"                  
[29] "establishmentMeans"            "primaryGrowthForm"            
[31] "primaryGrowthFormGroup"        "secondaryGrowthForms"         
[33] "secondaryGrowthFormGroups"     "stateConservation"            
[35] "protectedInNSW"                "sensitivityClass"             
[37] "TSProfileID"                   "countryConservation"          
[39] "highThreatWeed"                "widelyCultivatedNativeSpecies"
[41] "CAMBA"                         "JAMBA"                        
[43] "ROKAMBA"

In [6]:
dts %>% select(vernacularName) %>% slice_head(n=5)

vernacularName
<chr>
NA
NA
Desert Honey-myrtle
Flannel Flower
Sea Celery


We use the _dbWriteTable_ function to write the complete table into postgres

In [7]:
dbWriteTable(con,c("species","caps"),dts)

[1] TRUE

We then add an index for column _scientificName_, and a primary key on _speciesID_.

In [8]:
dbSendQuery(con,'CREATE INDEX name ON species.caps USING btree ("scientificName")')

<PostgreSQLResult>

In [5]:
dbSendQuery(con,'CREATE INDEX code ON species.caps USING btree ("currentScientificNameCode")')

<PostgreSQLResult>

In [9]:
dbSendQuery(con,'ALTER TABLE species.caps ADD PRIMARY KEY ("speciesID")')

<PostgreSQLResult>

We can now disconnect from the database

In [6]:
dbDisconnect(con)

[1] TRUE